In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.cross_validation import KFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVC
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import NearestNeighbors
import quadratic_weighted_kappa
from sklearn.linear_model import Ridge
from sklearn.metrics import make_scorer
from sklearn.cross_validation import train_test_split
from sklearn.grid_search import GridSearchCV
import feature_generator
import xgboost as xgb
from scipy import optimize
import os.path
from NN import NN
from XgBoost import XGBoostModel
from sklearn.linear_model import Ridge

Using Theano backend.


In [2]:
dfTrain = pd.read_csv('train.csv')
dfTest = pd.read_csv('test.csv')    
train, test, labels = feature_generator.GetFeatures(dfTrain, dfTest, 100, True)
# train, test, labels = feature_generator.make_dataset(True, "mean", True, dfTrain, dfTest)

Medical_History_2
Medical_History_10
Scaling...


In [3]:
# # oldInd = train[train['Ins_Age'] > train['Ins_Age'].median()].index
# ind = train[train['Ins_Age'] <= train['Ins_Age'].median()].index
# train = train.loc[ind]
# # test = test.iloc[ind]
# labels = labels.loc[ind]

In [4]:
def WritePredictionsToFile(model, modelName):
    
    kf = KFold(len(train), 3)
    num = 1
    for train_index, test_index in kf:
        foldFile = 'fold%s.csv' % str(num)
        if os.path.isfile(foldFile):
            predictionsDF = pd.read_csv(foldFile)  
        else:
            predictionsDF = pd.DataFrame()
          
        xTrain = train.iloc[train_index].values
        yTrain = labels.iloc[train_index]      
        model.fit(xTrain, yTrain)
        trainPredictions = model.predict(xTrain)
        
        xValidate = train.iloc[test_index].values
        yValidate = labels.iloc[test_index]
        predictions = model.predict(xValidate)
        predictionsDF[modelName] = predictions
        
        print quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, yValidate)
        print quadratic_weighted_kappa.quadratic_weighted_kappa(trainPredictions, yTrain)
        
#         youngTrainPredictions = [trainPredictions[i] for i in range(len(xTrain)) if i in youngInd]
#         oldTrainPredictions = [trainPredictions[i] for i in range(len(trainPredictions)) if i in oldInd]
#         youngYTrain = yTrain.iloc[youngInd] 
                        
        predictionsDF.to_csv(path_or_buf=foldFile, index=False)
   
        predictionsFile = 'testPredictions%s.csv' % str(num)
        if os.path.isfile(predictionsFile):
            testDF = pd.read_csv(predictionsFile)   
        else:
            testDF = pd.DataFrame()
            
        xTest = test.values
        testPredictions = model.predict(xTest)
        testDF[modelName] = testPredictions
        testDF.to_csv(path_or_buf=predictionsFile, index=False)
    
        num += 1

In [5]:
# WritePredictionsToFile(LogisticRegression(), 'LogisticRegression')
# WritePredictionsToFile(XGBoostModel(700, 10, 0.025, 0.65, "reg:linear"), 'XGBoostRegLin')

# WritePredictionsToFile(XGBoostModel(0.3, 1, 0, 700), 'XGBoostLinear')
#  __init__(self, max_depth, eta, colsample_bytree, objective):
# WritePredictionsToFile(BaggingRegressor(base_estimator=DecisionTreeRegressor(), n_estimators=20), 'BaggingDescisionTrees_n_estimators=20')
WritePredictionsToFile(BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20, max_features=0.9, max_samples=1.0), 'BaggingDescisionTreeClassifiers_n_estimators=20')
# WritePredictionsToFile(BaggingRegressor(base_estimator=LinearRegression(), n_estimators=10), 'BaggingLinearRegression_n_estimators=10')

0.520426724009
0.996979898227
0.516454884274
0.996442061475
0.526019188949
0.997765713136


In [ ]:
# dfTrain = pd.read_csv('train.csv')
# dfTest = pd.read_csv('test.csv')    
# train, test, labels = feature_generator.GetFeatures(dfTrain, dfTest, 100)
# train, test, labels = feature_generator.make_dataset(True, "mean", True, dfTrain, dfTest)
# WritePredictionsToFile(NN(inputShape = train.shape[1], layers = [100, 50], dropout = [0.5, 0.5], activation='sigmoid', loss='mae', optimizer = 'adadelta', init = 'glorot_normal', nb_epochs = 8), 'Keras')

In [21]:

scorer = make_scorer(quadratic_weighted_kappa.quadratic_weighted_kappa)
# print len(features)
# print len(dummyVariables)

In [22]:
def qwk_scorer(estimator, X, Y):
    predictions = np.clip(estimator.predict(X), 1, 8)
    return quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, Y)

In [28]:
# bestRandomForest = GetBestModel(lambda: RandomForestRegressor(n_estimators=10, max_depth=10), features) 
# bestRandomForest = GetBestModel(lambda: AdaBoostRegressor(LinearRegression()), features) 
# bestCombinedModel = GetBestModel(GenerateNewCombinedModel, features, 3)

X_train, X_test, y_train, y_test = train_test_split(train, labels, test_size=0.20, random_state=0)
bcf = BaggingClassifier(base_estimator=DecisionTreeClassifier(), n_estimators=20, max_features=0.9, max_samples=1.0)
bcf.fit(X_train, y_train)
# Set the parameters by cross-validation
tuned_parameters = [{'n_estimators': [15, 20, 25], 'max_samples': [0.9, 1.0], 'max_features': [0.9, 1.0]}]
# clf = GridSearchCV(SVR(kernel='rbf', max_iter=1000, epsilon=0.49, tol=0.01, verbose=True), tuned_parameters, cv=3, scoring=qwk_scorer)
# clf = GridSearchCV(BaggingClassifier(base_estimator=DecisionTreeClassifier()), tuned_parameters, cv=3, scoring=qwk_scorer)
# clf.fit(X_train, y_train)
# SVR(kernel='rbf', max_iter=1, , tol=0.01, verbose=True)  
#C=5, g=0.1
# TestQWK: 0.24807920235173586
#C=2, g=0.05, qwk = 0.21
#C=5, g=0.05, qwk = 0.25465662455597715
# do C 

In [29]:
print clf.best_estimator_

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=0.9,
         max_samples=1.0, n_estimators=20, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)


In [30]:
bcp = bcf.predict(X_test)
print quadratic_weighted_kappa.quadratic_weighted_kappa(bcp, y_test)

0.525540312378


In [ ]:
print clf.best_params_
clfPredictions = np.clip(clf.predict(X_test), 1, 8)

# dataPoints = list()

# folds = (2, 5, 10, 20)
# for K in folds:
#     _, testQwk, trainQwk = GetBestModel(GenerateNewCombinedModel, features, K)
#     dataPoints.append((K, testQwk, trainQwk))

In [ ]:
print np.min(clfPredictions)
print np.max(clfPredictions)
print clfPredictions

In [ ]:
quadratic_weighted_kappa.quadratic_weighted_kappa(clfPredictions, y_test)

In [ ]:
# import matplotlib.pyplot as plt
# from matplotlib.font_manager import FontProperties
# fontProps = FontProperties()
# fontProps.set_size('small')


# plt.figure(1)
# train_data = [point[2] for point in dataPoints]
# test_data = [point[1] for point in dataPoints]
# plt.plot(folds, train_data, label="Training Data")
# plt.plot(folds, test_data, label="Test Data")
# plt.title("Learning Curve for Logistic Regression")
# plt.xlabel("Training Set Size")
# plt.ylabel("Classification Accuracy")
# plt.legend(prop = fontProps)
# x1,x2,y1,y2 = plt.axis()
# plt.axis((x1,x2,y1,1.1))
# plt.show()

# print dataPoints

In [ ]:
trainPredictions = bestCombinedModel[0].predict(dfTrain[features].values)
trainPredictions = np.rint(trainPredictions).astype(int)
print quadratic_weighted_kappa.quadratic_weighted_kappa(trainPredictions, dfTrain.Response)

In [ ]:
bestPredictions = np.rint(bestCombinedModel[0].predict(dfTest[features].values)).astype(int)
bestPredictions = np.rint(bestPredictions).astype(int)
dfTest['Predictions'] = bestPredictions
print np.max(bestPredictions)

In [ ]:
dfTest.to_csv(path_or_buf='Combo.csv', columns=['Id', 'Predictions'], index=False, header=['Id', 'Response'])

In [ ]:
features = ['Ins_Age', 'BMI']
print len(dfTrain)
print len(dfTest)

In [ ]:
from scipy.stats import pearsonr
import matplotlib.pyplot as plt

keywords = ['Medical_Keyword_' + str(i) for i in range(1, 49)]
# print dfTrain[keywords].sum(axis=1)

# for keyword in keywords:
#     print pearsonr(dfTrain[keyword], dfTrain.Response)
#     break

# uniqueValues = dfTest['InsuredInfo_7'].unique()
# for i in range(len(uniqueValues)):
#     arr = dfTrain['InsuredInfo_7'].apply(lambda x: x == uniqueValues[i])
    

for column in ['Product_Info_4', 'Ins_Age', 'Ht', 'Wt', 'BMI', 'Employment_Info_1', 'Employment_Info_4', 'Employment_Info_6', 'Insurance_History_5', 'Family_Hist_2', 'Family_Hist_3', 'Family_Hist_4', 'Family_Hist_5']:
   
    if dfTrain[column].isnull().sum():
        print column
        print pearsonr(dfTrain[dfTrain[column].notnull()][column], dfTrain[dfTrain[column].notnull()].Response)
        print dfTrain[column].median()
        print len(dfTrain[dfTrain.Response == 8])
        print len(dfTrain)
#         print dfTrain[column]
#         plt.plot(dfTrain[dfTrain[column].notnull()][column], dfTrain[dfTrain[column].notnull()].Response)
        break
# plt.show()
# print dfTest['InsuredInfo_4'].unique()
# print dfTrain['InsuredInfo_6'].apply(lambda x: x == 1)
print dfTest['Medical_History_32'].isnull().sum()
            
print dfTrain['Medical_History_32'].median()
print dfTrain[dfTrain['Family_Hist_2'].notnull()]['Family_Hist_2'].max()
# print dfTrain['Medical_History_32'].null().sum()
# print dfTest[dfTest['Medical_History_32'].notnull()]['Medical_History_32'].median()
# print len(pd.concat([dfTest[dfTest['Medical_History_32'].notnull()]['Medical_History_32'], dfTrain[dfTrain['Medical_History_32'].notnull()]['Medical_History_32']]))
# print pearsonr(dfTrain[dfTrain['Medical_History_32'].notnull()]['Medical_History_32'], dfTrain[dfTrain['Medical_History_32'].notnull()]['Response'])

In [ ]:
neighborModel = GetBestModel(lambda: NearestNeighbors(), features)

In [ ]:
nearestNeighbors = neighborModel.kneighbors(dfTrain[features].values)

In [ ]:
predictions = np.zeros(len(dfTrain))
for i in range(len(dfTrain)):
#     predictions[i] = dfTrain.iloc[nearestNeighbors[1][i]].Response.mean()
    responses = dfTrain.iloc[nearestNeighbors[1][i]].Response
    weights = nearestNeighbors[0][i][4] - nearestNeighbors[0][i]
    predictions[i] = np.sum(responses * weights) / np.sum(weights)

In [ ]:
predictions = np.rint(predictions)
print quadratic_weighted_kappa.quadratic_weighted_kappa(predictions, dfTrain.Response)
print np.min(predictions)

In [ ]:
import scipy.stats as stats
lowerBound = np.mean(dfTrain['BMI']) - np.std(dfTrain['BMI'])
upperBound = np.mean(dfTrain['BMI']) + np.std(dfTrain['BMI'])
mean = np.mean(dfTrain['BMI'])
stats.normaltest(dfTrain['BMI'])

bmiScore = dfTrain['BMI'].apply(lambda x: max(0, lowerBound - x) if x < mean else max(0, x - upperBound))

model = LinearRegression(normalize=True)
features = ['Ins_Age', 'Wt', 'BMI']
X = np.ndarray((59381, 9))
X[:,0] = dfTrain['Ins_Age'].values
X[:,1] = dfTrain['BMI'].values
X[:,2] = dfTrain['Wt'].values
X[:,3] = dfTrain['Ins_Age'] * dfTrain['Wt']
X[:,4] = dfTrain['Ins_Age'] * dfTrain['BMI']
X[:,5] = dfTrain['Ins_Age'] * dfTrain['Ins_Age']
X[:,6] = dfTrain['Wt'] * dfTrain['Wt']
X[:,7] = dfTrain['Wt'] * dfTrain['BMI']
X[:,8] = dfTrain['Wt'] * dfTrain['Ins_Age']

model.fit(X, dfTrain['Response'])

In [ ]:
class CutPointOptimizer:
    
    def __init__(self, predicted, actual):
        self.predicted = predicted
        self.actual = actual

    def qwk(self, cutPoints):
        transformedPredictions = np.searchsorted(cutPoints, self.predicted) + 1            
        return -1 * quadratic_weighted_kappa.quadratic_weighted_kappa(transformedPredictions, self.actual)

initialCutPoints = np.array([1.5, 2.5, 3.5, 4.5, 5.5, 6.5, 7.5])

In [ ]:

# Product_Info_2
# Product_Info_3
# Employment_Info_2
# InsuredInfo_3
# Medical_History_2
# Medical_History_10

var = 'Product_Info_3'
for val in dfTrain[var].unique():
    hits = dfTrain[dfTrain[var] == val]
    print '%s, %s, %s' % (val, len(hits), np.mean(hits.Response))